# Model transforming a sequence of digits into a number
- input: sequence of three digits
- output: number in range(0,DATASE_SIZE)


Examples: 
- input: sequence [2,3,4] output: 234
- input: sequence [0,0,4] output: 4


In [1]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import numpy as np

## Model

In [2]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1)))
model.add(Dense(1))
model.summary()

model.compile(loss='mean_absolute_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


## Dataset creation

In [3]:
DATASET_SIZE=256

samples = []
labels = []

for i in range(DATASET_SIZE):
    labels.append(i)
    samples.append( list(str(i).rjust(3,'0')) )

samples = np.array(samples,dtype=float)
samples = np.expand_dims(samples,axis=2)
labels = np.array(labels,dtype=int)

print("Sample (input):\n",samples[123])
print("Label (output):",labels[123])
print("Samples size",samples.shape)
print("Labels size",labels.shape)

Sample (input):
 [[1.]
 [2.]
 [3.]]
Label (output): 123
Samples size (256, 3, 1)
Labels size (256,)


In [4]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 128  test samples 128


In [6]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print()
    print("Final epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
          format(num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model()
print("Done")

Training with 128 samples 100 epochs and batch_size= 32

Final epochs: 100 - loss=74.367, loss improvement=58.344
Average error: 70.903

Final epochs: 200 - loss=41.141, loss improvement=32.779
Average error: 38.815

Final epochs: 300 - loss=23.946, loss improvement=16.976
Average error: 21.817

Final epochs: 400 - loss=12.737, loss improvement=11.042
Average error: 10.942

Final epochs: 500 - loss= 6.355, loss improvement= 6.224
Average error:  5.023

Final epochs: 600 - loss= 2.982, loss improvement= 3.309
Average error:  2.537

Final epochs: 700 - loss= 1.406, loss improvement= 1.389
Average error:  1.246

Final epochs: 800 - loss= 0.687, loss improvement= 0.597
Average error:  0.691

Final epochs: 900 - loss= 0.487, loss improvement= 0.115
Average error:  0.765

Final epochs: 1000 - loss= 0.309, loss improvement= 0.308
Average error:  0.353
Done


In [7]:
def check_model(verbose=0,show_training=1):
    pred = model.predict(samples)
    for i in range(len(pred)):
        if(not show_training and i in trainLabels): continue
        p = pred[i][0]
        l = labels[i]
        if(verbose==1):
            train=''
            if i in trainLabels: train='[T]'
            print("{:2} > {:6.2f}, error ={:6.2f} {}".format(l,p,abs(p-l),train))
    avg_error = np.mean(np.absolute(pred-np.expand_dims(labels,axis=1)))
    print('Average error: {:6.3f}'.format(avg_error))
    return(avg_error)    
check_model(1)

 0 >   4.52, error =  4.52 
 1 >   4.11, error =  3.11 
 2 >   3.84, error =  1.84 
 3 >   3.77, error =  0.77 
 4 >   4.02, error =  0.02 [T]
 5 >   4.71, error =  0.29 
 6 >   5.97, error =  0.03 [T]
 7 >   7.74, error =  0.74 
 8 >   9.78, error =  1.78 
 9 >  11.89, error =  2.89 
10 >  10.11, error =  0.11 
11 >  10.98, error =  0.02 [T]
12 >  11.91, error =  0.09 [T]
13 >  12.89, error =  0.11 
14 >  13.89, error =  0.11 [T]
15 >  14.92, error =  0.08 
16 >  15.93, error =  0.07 
17 >  16.91, error =  0.09 
18 >  17.86, error =  0.14 [T]
19 >  18.83, error =  0.17 [T]
20 >  19.95, error =  0.05 [T]
21 >  20.90, error =  0.10 
22 >  21.89, error =  0.11 
23 >  23.00, error =  0.00 [T]
24 >  24.05, error =  0.05 [T]
25 >  24.96, error =  0.04 
26 >  25.87, error =  0.13 
27 >  26.82, error =  0.18 [T]
28 >  27.80, error =  0.20 [T]
29 >  28.70, error =  0.30 
30 >  30.12, error =  0.12 
31 >  31.02, error =  0.02 [T]
32 >  31.91, error =  0.09 [T]
33 >  32.94, error =  0.06 [T]
34 

0.3531241286545992

In [ ]:
model.predict([[[2],[2],[1]]])

In [ ]:
model.save('model_135_999.h5')